# Improve retrieval performance by Fine-tuning embedding model

Another way to improve retriever performance is to fine-tune the embedding model itself. Fine-tuning the embedding model can help in learning better representations for the documents and queries in the dataset. This can be particularly useful when the dataset is very different from the pre-trained data used to train the embedding model.

In [2]:
%pip install llama-index-llms-openai llama-index-embeddings-openai llama-index-finetuning llama-index-readers-file scikit-learn llama-index-embeddings-huggingface llama-index-vector-stores-lancedb pyarrow==12.0.1 -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.5/25.5 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.4/328.4 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [2]:
# For eval utils
!git clone https://github.com/lancedb/ragged.git
!cd ragged && pip install .


Cloning into 'ragged'...
remote: Enumerating objects: 160, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 160 (delta 70), reused 125 (delta 41), pack-reused 0
Receiving objects: 100% (160/160), 38.15 KiB | 5.45 MiB/s, done.
Resolving deltas: 100% (70/70), done.
Processing /content/ragged
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━

## The dataset
The dataset we'll use is a synthetic QA dataset generated from LLama2 review paper. The paper was divided into chunks, with each chunk being a unique context. An LLM was prompted to ask questions relevant to the context for testing a retreiver.
The exact code and other utility functions for this can be found in [this](https://github.com/lancedb/ragged) repo


In [3]:
!wget https://raw.githubusercontent.com/AyushExel/assets/main/data_qa.csv

--2024-07-09 14:17:49--  https://raw.githubusercontent.com/AyushExel/assets/main/data_qa.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 680439 (664K) [text/plain]
Saving to: ‘data_qa.csv’

data_qa.csv         100%[===================>] 664.49K  --.-KB/s    in 0.04s   

2024-07-09 14:17:49 (17.6 MB/s) - ‘data_qa.csv’ saved [680439/680439]



In [1]:
import pandas as pd

data = pd.read_csv("data_qa.csv")

## Pre-processing
Now we need to parse the context(corpus) of the dataset as llama-index text nodes.  

In [2]:
from pathlib import Path
from llama_index.core.node_parser import SentenceSplitter
from llama_index.readers.file import PagedCSVReader

def load_corpus(file, verbose=False):
    if verbose:
        print(f"Loading files {file}...")

    loader = PagedCSVReader(encoding="utf-8")
    docs = loader.load_data(file=Path(file))

    if verbose:
        print(f"Loaded {len(docs)} docs")

    parser = SentenceSplitter()
    nodes = parser.get_nodes_from_documents(docs, show_progress=verbose)

    if verbose:
        print(f"Parsed {len(nodes)} nodes")

    return nodes

In [3]:
import pandas as pd

df = pd.read_csv("data_qa.csv", index_col=0)

In [4]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."

Split into train and validation sets. We'll use the original df for val as that has different queries generated via a different prompt.


In [5]:
from sklearn.model_selection import train_test_split

# Randomly shuffle df
df = df.sample(frac=1, random_state=42)

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

train_df.to_csv("train_data_qa.csv", index=False)
val_df.to_csv("val_data_qa.csv", index=False)

In [6]:
train_nodes = load_corpus("train_data_qa.csv", verbose=True)
val_nodes = load_corpus("val_data_qa.csv", verbose=True)

Loading files train_data_qa.csv...
Loaded 176 docs


Parsing nodes:   0%|          | 0/176 [00:00<?, ?it/s]

Parsed 227 nodes
Loading files val_data_qa.csv...
Loaded 44 docs


Parsing nodes:   0%|          | 0/44 [00:00<?, ?it/s]

Parsed 53 nodes


### Generate the query from context from training


In [7]:
from llama_index.finetuning import generate_qa_embedding_pairs
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset

In [8]:
from llama_index.llms.openai import OpenAI


train_dataset = generate_qa_embedding_pairs(
    llm=OpenAI(model="gpt-3.5-turbo"), nodes=train_nodes, verbose=False
)
val_dataset = generate_qa_embedding_pairs(
    llm=OpenAI(model="gpt-3.5-turbo"), nodes=val_nodes, verbose=False
)

train_dataset.save_json("train_dataset.json")
val_dataset.save_json("val_dataset.json")

227it [00:00, ?it/s]
227it [00:00, ?it/s]


In [9]:
# Load again
train_dataset = EmbeddingQAFinetuneDataset.from_json("train_dataset.json")

val_dataset = EmbeddingQAFinetuneDataset.from_json("val_dataset.json")

## Fine-tune the embedding model

In [10]:
import torch
from llama_index.finetuning import SentenceTransformersFinetuneEngine

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

finetune_engine = SentenceTransformersFinetuneEngine(
    train_dataset,
    model_id="BAAI/bge-small-en-v1.5",
    model_output_path="tuned_model",
    val_dataset=val_dataset,
    device=device
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
finetune_engine.finetune( )


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/46 [00:00<?, ?it/s]

Iteration:   0%|          | 0/46 [00:00<?, ?it/s]

In [12]:
embed_model = finetune_engine.get_finetuned_model()


## Evaluate on Hit-rate


In [13]:
from ragged.dataset import CSVDataset, SquadDataset
from ragged.rag import llamaIndexRAG
from ragged.metrics.retriever.hit_rate import HitRate
from ragged.search_utils import QueryType


def evaluate_vector(
    dataset,
    embed_model_name_or_path,
    top_k=5,
):
  dataset = CSVDataset(dataset)

  hit_rate = HitRate(dataset, embed_model_kwarg={"name": embed_model_name_or_path})

  print(hit_rate.evaluate(top_k, query_type=QueryType.VECTOR))


def evaluate_all(
    dataset,
    embed_model_name_or_path,
    reranker,
    top_k=5,
):
  dataset = CSVDataset(dataset)
  hit_rate = HitRate(dataset, embed_model_kwarg={"name": embed_model_name_or_path}, reranker=reranker)

  print(hit_rate.evaluate(top_k, query_type=QueryType.ALL))


In [14]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from sentence_transformers import SentenceTransformer
from pathlib import Path


def evaluate_IR(
    dataset,
    model_id,
    name,
):
    corpus = dataset.corpus
    queries = dataset.queries
    relevant_docs = dataset.relevant_docs

    output_path = "./results/"
    Path(output_path).mkdir(exist_ok=True, parents=True)

    evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs, name=name)
    model = SentenceTransformer(model_id)

    return evaluator(model, output_path=output_path)

In [15]:
from lancedb.rerankers import CohereReranker, LinearCombinationReranker


#linear_combination_reranker = LinearCombinationReranker()
cohere_reranker = CohereReranker(api_key="...")

#evaluate_all("data_qa.csv", "BAAI/bge-small-en-v1.5", linear_combination_reranker)
hit_rate_bge_cohere = evaluate_all("data_qa.csv", "BAAI/bge-small-en-v1.5", cohere_reranker)


INFO:lancedb:Adding 110 documents to LanceDB, in 1 batches of size 110


Adding 110 documents to LanceDB, in 1 batches of size 110


Adding batch to LanceDB: 100%|██████████| 110/110 [00:00<00:00, 159810.68it/s]
INFO:lancedb:Adding batch 0 to LanceDB


Adding batch 0 to LanceDB


INFO:lancedb:created table with length 110


...

Hit rate for rerank_fts: 0.6727272727272727


INFO:lancedb:Evaluating query type: hybrid


Evaluating query type: hybrid


  0%|          | 0/220 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/lancedb/rerankers/cohere.py:88: FutureWarning: promote has been superseded by promote_options='default'.
  combined_results = self.merge_results(vector_results, fts_results)
  0%|          | 1/220 [00:00<00:46,  4.74it/s]/usr/local/lib/python3.10/dist-packages/lancedb/rerankers/cohere.py:88: FutureWarning: promote has been superseded by promote_options='default'.
  combined_results = self.merge_results(vector_results, fts_results)
  1%|          | 2/220 [00:00<00:45,  4.80it/s]/usr/local/lib/python3.10/dist-packages/lancedb/rerankers/cohere.py:88: FutureWarning: promote has been superseded by promote_options='default'.
  combined_results = self.merge_results(vector_results, fts_results)
  1%|▏         | 3/220 [00:00<00:43,  5.00it/s]/usr/local/lib/python3.10/dist-packages/lancedb/rerankers/cohere.py:88: FutureWarning: promote has been superseded by promote_options='default'.
  combined_results = self.merge_r

Hit rate for hybrid: 0.759090909090909
vector=0.6409090909090909 fts=0.5954545454545455 rerank_vector=0.6772727272727272 rerank_fts=0.6727272727272727 hybrid=0.759090909090909


In [16]:
#evaluate_all("data_qa.csv", "tuned_model/", linear_combination_reranker)
evaluate_all("data_qa.csv", "tuned_model/", cohere_reranker)



INFO:lancedb:Adding 110 documents to LanceDB, in 1 batches of size 110


Adding 110 documents to LanceDB, in 1 batches of size 110


Adding batch to LanceDB: 100%|██████████| 110/110 [00:00<00:00, 132388.36it/s]
INFO:lancedb:Adding batch 0 to LanceDB


Adding batch 0 to LanceDB


INFO:lancedb:created table with length 110


...

Hit rate for rerank_fts: 0.6727272727272727


INFO:lancedb:Evaluating query type: hybrid


Evaluating query type: hybrid


  0%|          | 0/220 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/lancedb/rerankers/cohere.py:88: FutureWarning: promote has been superseded by promote_options='default'.
  combined_results = self.merge_results(vector_results, fts_results)
  0%|          | 1/220 [00:00<00:42,  5.10it/s]/usr/local/lib/python3.10/dist-packages/lancedb/rerankers/cohere.py:88: FutureWarning: promote has been superseded by promote_options='default'.
  combined_results = self.merge_results(vector_results, fts_results)
  1%|          | 2/220 [00:00<00:45,  4.77it/s]/usr/local/lib/python3.10/dist-packages/lancedb/rerankers/cohere.py:88: FutureWarning: promote has been superseded by promote_options='default'.
  combined_results = self.merge_results(vector_results, fts_results)
  1%|▏         | 3/220 [00:01<01:38,  2.21it/s]/usr/local/lib/python3.10/dist-packages/lancedb/rerankers/cohere.py:88: FutureWarning: promote has been superseded by promote_options='default'.
  combined_results = self.merge_r

Hit rate for hybrid: 0.7727272727272727
vector=0.6727272727272727 fts=0.5954545454545455 rerank_vector=0.740909090909091 rerank_fts=0.6727272727272727 hybrid=0.7727272727272727
